In [1]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [3]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    for z in range(cells.shape[2]-1):
        for y in range(cells.shape[1]-1):
            for x in range(cells.shape[0]-1):
                case_nums = getCaseNum(x, y, z, thres, cells)
                triangle = []
                for case_num in case_nums:
                    if case_num == -1: 
                        continue
                    edge = CaseNum2EdgeOffset[case_num]
                    p1 = (x + edge[0], y + edge[1], z + edge[2])
                    p2 = (x + edge[3], y + edge[4], z + edge[5])

                    v1 = cells[p1]
                    v2 = cells[p2]
                    t = (thres - v1) / (v2 - v1) if v2 != v1 else 0.5
                    
                    interp_point = (
                        p1[0] + t*(p2[0]-p1[0]),
                        p1[1] + t*(p2[1]-p1[1]),
                        p1[2] + t*(p2[2]-p1[2])
                    )
                    if interp_point not in vertex_array:
                        vertex_array[interp_point] = len(vertex_array)
                    
                    triangle.append(vertex_array[interp_point])
                for i in range(0, len(triangle)-2, 3):
                    face_array.append([triangle[i], triangle[i+1], triangle[i+2]])

    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.keys())
    return np.array(vertex_array), np.array(face_array, dtype=np.int64)

In [4]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
3.3300585746765137 s

Time taken by algorithm
----------------------------------------
3.3578052520751953 s
